In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats




ImportError: cannot import name '_is_pandas_dataframe' from 'matplotlib.cbook' (C:\Users\fuzzy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\matplotlib\cbook\__init__.py)

In [ ]:
data = pd.read_csv('nba_games.csv', index_col=0)
data = data.dropna()

data.describe()

### Let's observe our data. Hey, that looks like a normal distribution!!

In [ ]:
data.hist(figsize=(12, 8), bins=60)
plt.suptitle("Feature Distributions", fontsize=16)
plt.show()

In [ ]:


data['ratings'] = data['rating_a'].values + data['rating_b'].values
data = data[data['ratings'] >= 0]
ratings = data['ratings']

total_scores = data['total_score'].values

print(np.mean(ratings))
print(np.mean(total_scores))
print(np.var(total_scores))

#variance much higher than mean, will take this into account when selecting a model...



In [ ]:
plt.scatter(data['rating_a'] - data['rating_b'], data['score_a'], s=5)
plt.show()

In [ ]:
from scipy.stats import poisson, nbinom

ts_mean = np.mean(total_scores)
ts_var = np.var(total_scores)

#plot data
sns.histplot(total_scores, bins=40, kde=False, color="blue", stat="density", label="Distribution of Total Points")
x = np.arange(min(total_scores), max(total_scores))


#get the pmf of the poisson distribution
poisson_pmf = poisson.pmf(x, mu=ts_mean)

#plot it
plt.plot(x, poisson_pmf, 'g-', label='Poisson')



r_est = (ts_mean ** 2) / (ts_var - ts_mean)
p_est = ts_mean / ts_var
nb_pmf = nbinom.pmf(x, r_est, p_est)
plt.plot(x, nb_pmf, 'r-', label="Negative Binomial")

plt.legend()
plt.xlabel('Total Score')
plt.ylabel('Density')
plt.title('Histogram with Distribution Fits')
plt.show()


This looks pretty obvious that the negative binomial distribution is a better fit, and we have some overdispersion so it makes sense to me to use a negative binomial model. However, I am seeing a lot of resources online that say that binomial models are only good for count data of one type of occurrence, and total scores which includes multiple types of points doesn't fall under that criteria (I think).